In [43]:
from IPython.display import display, Markdown
import os
from LS_AMG_RAG.data_snythesis import prompt_utils
import google.generativeai as palm
import time

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
# chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="./")
# chroma_client = chromadb.HttpClient(host='localhost', port=8000)
gemini = prompt_utils.Gemini()
google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.environ['GEMINI_API_KEY'])

In [44]:
chroma_client.heartbeat()

1711085828386036100

In [45]:
try:
    collection = chroma_client.get_or_create_collection(
        name="my_collection",
        embedding_function=google_ef,
        metadata={"hnsw:space": "cosine"})
except:
    collection = chroma_client.create_collection(
        name="my_collection",
        embedding_function=google_ef,
        metadata={"hnsw:space": "cosine"})
    documents = []
    metadata = []
    ids = []

    for root, dirs, files in os.walk("../data"):
        for file in files:
            if file.endswith(".md"):
                category = root.split('\\')[-1]
                with open(os.path.join(root, file), "r") as f:
                    file_contents = f.read()
                    documents.append(file_contents)
                    metadata.append({
                        "type": category,
                    })
                    ids.append(f"{category}_{file}")

    collection.add(
        documents=documents,
        metadatas=metadata,
        ids=ids,
    )

else:
    print(f"Total number of documents in the collection: {len(collection.get()['ids'])}")

Total number of documents in the collection: 74


In [46]:
# documents = []
# metadata = []
# ids = []

# for root, dirs, files in os.walk("../data"):
#     for file in files:
#         if file.endswith(".md"):
#             category = root.split('\\')[-1]
#             with open(os.path.join(root, file), "r") as f:
#                 file_contents = f.read()
#                 documents.append(file_contents)
#                 metadata.append({
#                     "type": category,
#                 })
#                 ids.append(f"{category}_{file}")

# collection.add(
#     documents=documents,
#     metadatas=metadata,
#     ids=ids,
# )

In [47]:
top_k = {
    1: [],
    3: [],
    5: [],
    10: [],
}

queries = [
    {'query': "What is Instagram's current business proposal?",
     'file': "Business Proposal.md"},
    {'query': "What is the marketing plan for Instagram?",
     'file': "Marketing Plan.md"},
    {'query': "What information does the progress report of Instagram contain?",
     'file': "Progress Report.md"},
    {'query': "Who are the members of Instagram's board of directors?",
     'file': "Board of Directors.md"},
    {'query': "What are the diversity and inclusion initiatives implemented by Instagram?",
     'file': 'Diversity, Equity, and Inclusion.md'},
    {'query': "What is the Marketing Objective for Influencer Collaboration Services?",
     'file': 'Marketing Plan.md'},
    {'query': "Who is the target audience of Content Creation and Curation Services?",
     'file': 'Marketing Plan.md'},
    {'query': "What is the financial update for the Reels Optimization Project?",
     'file': 'Progress Report.md'},
    {'query': "Give me details about the progress report of the Stories Upgrade project.",
     'file': 'Progress Report.md'},
    {'query': "Compare the progress report of the Feed Redesign and Stories Upgrade project and draw a conclusion on the information.",
     'file': 'Progress Report.md'}
]

step_times = {
    'retrieval': [],
    'gen': [],
    'total': [],
}

In [48]:

for idx, query in enumerate(queries):
    print(f"Query {idx+1}: {query['query']}")
    total_start_time = time.time()
    retrieval_start_time = time.time()
    results = collection.query(
        query_texts=query['query'],
        n_results=10,
    )

    for k in top_k.keys():
        top_k[k].append(any(query['file'] in x for x in results['ids'][0][:k]))

    retrieval_end_time = time.time()
    print(f"True document: {query['file']}")
    print(f"Document retrieved: {results['ids'][0][0]}")
    print(f"Retrieval time: {retrieval_end_time - retrieval_start_time:.2f} seconds")


    metaprompt = """You are a helpful and informative bot that answers questions using text from the reference document included below. \
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
strike a friendly and converstional tone. \
Use your own knowledge base in addition to the information provided in the document to answer the question. \
Make relevant assumptions and use your best judgement to answer the question. \
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

  ANSWER:
"""

    gen_start_time = time.time()
    gemini_result = gemini.send_message(message=metaprompt.format(query=queries[0], relevant_passage=results['documents'][0][0])).text
    gen_end_time = time.time()
    total_end_time = time.time()

    step_times['retrieval'].append(retrieval_end_time - retrieval_start_time)
    step_times['gen'].append(gen_end_time - gen_start_time)
    step_times['total'].append(total_end_time - total_start_time)
    
    display(Markdown(gemini_result))
    print(f"Gemini time: {gen_end_time - gen_start_time:.2f} seconds")
    print(f"Total time: {total_end_time - total_start_time:.2f} seconds")
    print("\n-------------------\n")

print("Results:")
print("Top@K:")
for k in top_k.keys():
    print(f"Top@{k}: {(sum(top_k[k]) * 100) / len(top_k[k]):.2f}%")

print("\n")

print("Average Times:")
print(f"Retrieval time: {sum(step_times['retrieval']) / len(step_times['retrieval']):.2f} seconds")
print(f"Generation time: {sum(step_times['gen']) / len(step_times['gen']):.2f} seconds")
print(f"Total time: {sum(step_times['total']) / len(step_times['total']):.2f} seconds")

Query 1: What is Instagram's current business proposal?


True document: Business Proposal.md
Document retrieved: business_docs_Business Proposal.md
Retrieval time: 1.69 seconds


Instagram proposes a suite of marketing services to businesses, encompassing social media management, influencer collaborations, and content creation. Their social media marketing services involve strategic content creation, targeted ad campaigns, audience analysis, and performance tracking. Instagram also offers influencer collaboration services, leveraging their platform's influencer community to connect brands with relevant influencers for impactful partnerships. Additionally, they provide content creation and curation services, assisting businesses in developing visually appealing and engaging content for their social media presence.

Gemini time: 3.14 seconds
Total time: 4.83 seconds

-------------------

Query 2: What is the marketing plan for Instagram?
True document: Marketing Plan.md
Document retrieved: business_docs_Marketing Plan.md
Retrieval time: 0.41 seconds


Instagram's current business proposal revolves around three core offerings: social media marketing services, influencer collaboration services, and content creation and curation services. The social media marketing services encompass strategic content creation, targeted ad campaigns, audience analysis, and performance tracking to help businesses enhance their online presence and customer engagement. Instagram also offers influencer collaboration services, leveraging their platform's community of influencers to connect brands with relevant influencers for impactful partnerships. These collaborations aim to drive brand awareness, user engagement, and brand advocacy through authentic influencer endorsements. Additionally, Instagram's content creation and curation services assist businesses in developing visually appealing and engaging content for their social media presence, elevating brand storytelling and audience interaction.

...

Gemini time: 4.51 seconds
Total time: 4.92 seconds

-------------------

Query 9: Give me details about the progress report of the Stories Upgrade project.
True document: Progress Report.md
Document retrieved: business_docs_Progress Report.md
Retrieval time: 0.42 seconds


The provided document does not contain information about Instagram's current business proposal. It primarily focuses on a progress report of various projects within Instagram, such as Feed Redesign, Stories Upgrade, Direct Messaging Update, Explore Algorithm Enhancement, IGTV Expansion, and Reels Optimization. Without access to up-to-date information or insights into Instagram's business strategy, I cannot provide a response regarding their current business proposal.

Gemini time: 4.43 seconds
Total time: 4.85 seconds

-------------------

Query 10: Compare the progress report of the Feed Redesign and Stories Upgrade project and draw a conclusion on the information.
True document: Progress Report.md
Document retrieved: business_docs_Progress Report.md
Retrieval time: 0.42 seconds


I'm sorry, but the document you provided does not contain information about Instagram's current business proposal. The document focuses on a progress report of various projects within Instagram and does not provide insights into the company's business strategy or proposals.

Gemini time: 5.13 seconds
Total time: 5.55 seconds

-------------------

Results:
Top@K:
Top@1: 100.00%
Top@3: 100.00%
Top@5: 100.00%
Top@10: 100.00%


Average Times:
Retrieval time: 0.55 seconds
Generation time: 4.84 seconds
Total time: 5.39 seconds
